In [1]:
import spacy
import en_core_web_sm
import csv
import json
from spacy.matcher import Matcher
import re
import pandas as pd

In [2]:
nlp = en_core_web_sm.load()

In [3]:
#fitness_kg\Data_Scrape_Yoga\yoga\Extracted
bio_list = []
gym_dict = {'id':[],
                 'name':[], 'url':[], 'muscle':[],'type': [],
                 'equipment':[], 'mechanics':[],'force_type':[],'difficulty':[]
           ,'overview':[],'instructions':[],'tips':[],'secondary_muscle':[]}

with open('../fitness_kg/Extracted/Gym/exercise_group.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    
    gym_dict['id'].append(result['id'])
    gym_dict['name'].append(result['name'])
    gym_dict['url'].append(result['url'])
    gym_dict['muscle'].append(result['muscle'])
    gym_dict['type'].append(result['type'])
    gym_dict['equipment'].append(result['equipment'])
    gym_dict['mechanics'].append(result['mechanics'])
    gym_dict['instructions'].append(result['instructions'])
    gym_dict['force_type'].append(result['force_type'])
    gym_dict['difficulty'].append(result['difficulty'])
    gym_dict['overview'].append(result['overview'])
    gym_dict['tips'].append(result['tips'])
    gym_dict['secondary_muscle'].append(result['secondary_muscle'])    
    

In [4]:
exercise_group_df = pd.DataFrame.from_dict(gym_dict)

In [5]:
exercise_group_df.head()

,id,name,url,muscle,type,equipment,mechanics,force_type,difficulty,overview,instructions,tips,secondary_muscle
0,https://www.muscleandstrength.com/exercises/wi...,wide grip upright row,https://www.muscleandstrength.com/exercises/wi...,traps,strength,barbell,compound,pull,intermediate,,load up a barbell with the weight you want to ...,focus on keeping your elbows higher than your ...,"lats, shoulders"
1,https://www.muscleandstrength.com/exercises/ba...,barbell shrug,https://www.muscleandstrength.com/exercises/ba...,traps,strength,barbell,isolation,pull (bilateral),beginner,the barbell shrug is a great exercise for thos...,position the safeties just below waist height ...,looking slightly up while shrugging may enhanc...,"shoulders, upper back"
2,https://www.muscleandstrength.com/exercises/du...,dumbbell goblet squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (bilateral),beginner,the dumbbell goblet squat is a variation of th...,select a dumbbell and position it at chest hei...,if you struggle with squatting with a barbell ...,"abs, adductors, calves, glutes, hamstrings, lo..."
3,https://www.muscleandstrength.com/exercises/du...,dumbbell rear lunge,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),intermediate,the dumbbell reverse lunge is a dumbbell lunge...,set up with your feet shoulder width apart and...,reverse lunges are a more advanced progression...,"calves, glutes, hamstrings"
4,https://www.muscleandstrength.com/exercises/du...,dumbbell split squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),beginner,the dumbbell split squat is a dumbbell lunge v...,set up in a split stance position while grippi...,if you want to emphasize the quads during the ...,"abs, adductors, calves, glutes, hamstrings"


# Information Extraction

In [6]:
test = exercise_group_df['instructions'].values[26]
print(test)
print(exercise_group_df['name'].values[26])
doc = nlp(test)

setup in a supine position with your arms extended vertically and a partner standing over your chest holding a medicine ball.  as you partner pushes the medicine ball towards you, receive the ball and allow your arms to bend as your absorb the impact.  as soon as your elbows touch the floor, explode upwards, extend the elbows, and launch the medicine ball vertically.  your partner should catch the ball in mid flight and immediately direct it downward to complete another repetition.  repeat the entire sequence for the desired number of repetitions.
lying medicine ball power drop


In [7]:
# find_equipments(doc)

In [8]:
for ent in doc.ents:
            print(ent.text, ent.label_)

In [9]:
# for text in exercise_group_df['instructions'].values:
#     doc = nlp(text)
#     for ent in doc.ents:
#             print(ent.text, ent.label_)

In [10]:
def find_equipments(doc):
    equipment_patterns = [
               [{'POS':'VERB'},{'POS': 'DET'},{'POS': 'NOUN'}],  
                [{'POS':'VERB'},{'POS': 'ADP','OP':'+'},{'POS': 'DET'},{'POS': 'NOUN'}],
        [{'POS': 'SCONJ'},{'POS':'VERB'},{'POS': 'NOUN'}], 
             [{'POS':'VERB'},{'POS': 'DET'},{'POS':'NOUN','OP': '+'},{'POS': 'ADP'},{'POS': 'DET'},{'POS': 'NOUN'}],
        [{'POS':'VERB'},{'POS': 'ADP'},{'POS':'ADV'},{'POS': 'ADP'},{'POS': 'NOUN'},{'POS':'ADP'},{'POS': 'DET'}
         ,{'POS': 'ADJ'},{'POS': 'NOUN'}],
        [{'POS':'VERB'},{'POS': 'DET'},{'POS':'NOUN','OP': '+'}],
        [{'POS':'NOUN','OP':'+'}, {'POS':'ADJ'}, {'POS':'ADP'}, {'POS': 'DET'},{'POS':'NOUN'}]
        ]
    
    matcher = Matcher(nlp.vocab) 
    matcher.add("matching_2", equipment_patterns)
    matches = matcher(doc)
    matched_strings = []
    if len(matches)>0:
        for match_id, start, end in matches:
            
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = doc[start:end]  # The matched span
            req_word = span.text.split(" ")[-1]
            
            lemma_word = nlp(req_word)[0].lemma_
            
            matched_strings.append(lemma_word)
    
    return list(set(matched_strings))


def find_positions(doc):
    position_patterns = [
              [{'LEMMA': {"IN":['sit','stand','standing','supine', 'lay', 'lie']}}],
        ]
    
    matcher = Matcher(nlp.vocab) 
    matcher.add("matching_3", position_patterns)
    matches = matcher(doc)
    matched_strings = []
    if len(matches)>0:
        for match_id, start, end in matches:
            
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = doc[start:end]  # The matched span
            req_word = span.text.split(" ")[-1]
            
            lemma_word = nlp(req_word)[0].lemma_
            
            matched_strings.append(lemma_word)
    
    return list(set(matched_strings))

In [11]:

# for token in doc:
#     print(f'{token.text:{8}} {token.pos_:{6}} {token.tag_:{6}} {token.dep_:{6}} {spacy.explain(token.pos_):{20}} {spacy.explain(token.tag_)}')

In [12]:
# output_df = exercise_group_df[['id','instructions']]

In [13]:
extractions = []
i=0
temp_set = set()
equipment_set = set(['kettlebell','physioball','barbell','bench','bar','sled','dumbbell','chair','belt','cable','kettle'])
for url, instructions in exercise_group_df[['id','instructions']].values:
    cur_dir_dict = {}
    doc = nlp(instructions)
    
    cur_dir_dict['id'] = url
    
    ##Equipments
    data_extracted = find_equipments(doc)
    
    temp_set.update(data_extracted)
    req_equips = equipment_set.intersection(set(data_extracted))
    
    cur_dir_dict['equipment'] = list(req_equips)
    extractions.append(cur_dir_dict)
    ###Positions
    
    cur_dir_dict['position'] = find_positions(doc)

In [14]:
len(temp_set)

239

In [15]:
final_df = pd.DataFrame.from_dict(extractions)

In [16]:
final_df.head()

,id,equipment,position
0,https://www.muscleandstrength.com/exercises/wi...,"[bar, barbell]",[stand]
1,https://www.muscleandstrength.com/exercises/ba...,[bar],[stand]
2,https://www.muscleandstrength.com/exercises/du...,[dumbbell],[]
3,https://www.muscleandstrength.com/exercises/du...,[dumbbell],[stand]
4,https://www.muscleandstrength.com/exercises/du...,[dumbbell],[]


In [17]:
final_df.iloc[1073,:]

id           https://www.muscleandstrength.com/exercises/pl...
equipment                                                   []
position                                               [stand]
Name: 1073, dtype: object

In [18]:
final_df.to_csv("Data Extracted/Gym_info_extracted.csv", index = False)